In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

def gen_X(num_var: int, sample_size : int):
    data_dictionary = {}
    for i in range(num_var):
        var_name = "x" + f"{i + 1}"
        data_dictionary[var_name] = list(np.random.binomial(1, .5, sample_size))
    return pd.DataFrame(data_dictionary)


In [2]:
os.getcwd()

'/Users/gradcheckout/Lee/barcode/chapter_2_simulation'

In [3]:
# sys.path.append('/root/barcode/')
sys.path.append('/Users/gradcheckout/Lee/barcode/')
from BarcodeScanner import base_barcode

In [28]:
X = gen_X(3, 2000)

In [29]:
y = X.apply(lambda seq: 1+ seq.x1 + seq.x2 + seq.x2*seq.x3 + np.random.normal(scale = 1.4), axis = 1)

In [30]:
barcode = base_barcode(X, y)

In [32]:
import torch
from torch import nn

class Lasso_Barcode(nn.Module):
    def __init__(self, L_inv, num_variable):
        super().__init__()
        barcode_size = 2**num_variable
        L_inv = torch.from_numpy(L_inv.astype(float))
        self.embedding = nn.Embedding(barcode_size, barcode_size)
        self.embedding.weight = nn.Parameter(L_inv.to(torch.float32), requires_grad = False)
        self.linear = nn.Linear(barcode_size, 1, bias = False, dtype = torch.float32)

    def l1_reg(self):
        return torch.abs(self.linear.weight).sum()

    def forward(self, x):
        x = self.embedding(x)
        x = self.linear(x)
        return x, self.l1_reg()

In [33]:
input_tensor = torch.from_numpy(base_barcode.gen_barcode(X).reshape(-1))
input_tensor = input_tensor.to(torch.int32)

In [34]:
lasso = Lasso_Barcode(barcode.L, 3)

In [35]:
device = torch.device("cpu")
lasso = lasso.to(device)
input_tensor = input_tensor.to(device)

In [36]:
lasso.linear.weight

Parameter containing:
tensor([[-0.1574, -0.2820, -0.0563,  0.1462, -0.0582,  0.3386, -0.2914,  0.2416]],
       requires_grad=True)

In [37]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(lasso.parameters(), lr=0.01)


In [38]:
# input_tensor = torch.from_numpy(barcode.gen_barcode(X).reshape(-1))
# input_tensor = input_tensor.to(torch.int32)

alpha = 0.6


In [39]:
output_feature = torch.from_numpy(y.to_numpy().reshape(-1,1))
output_feature = output_feature.to(torch.float32)
output_feature = output_feature.to(device)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    optimizer.zero_grad()

    outputs, l1_reg = lasso(input_tensor)
    # loss = criterion(outputs, output_feature)

    loss = criterion(outputs, output_feature) + alpha * l1_reg  # Total loss with L1 regularization
    loss.backward()
    optimizer.step()
    # lasso.linear.weight = nn.Parameter((lasso.linear.weight[0] * lasso.linear.weight[0].abs().gt(0.01).to(torch.uint8)).reshape(1, -1))
    # optimizer = optim.SGD(lasso.parameters(), lr=0.001)


In [23]:
lasso.linear.weight[0].abs().sum()

tensor(2.7505, grad_fn=<SumBackward0>)

In [24]:
lasso.linear.weight[0].abs().sum()

tensor(2.7505, grad_fn=<SumBackward0>)

In [25]:
lasso.linear.weight[0].abs().lt(0.01).sum()

tensor(0)

In [26]:
lasso.linear.weight[0]

tensor([ 0.2624,  0.0494,  0.5110,  0.5294,  0.5094,  0.4343,  0.4088, -0.0458],
       grad_fn=<SelectBackward0>)

Since it frequently accounters a "out of memory" problem, I am writing a short script for data loader. In this step, the fabric data will have a temporary file in csv and the data loader will read the temp file over.

In [27]:
X

,x1,x2,x3
0,0,0,1
1,0,0,1
2,1,1,0
3,0,0,1
4,1,1,0
...,...,...,...
495,0,1,0
496,0,1,0
497,0,1,1
498,1,0,1
